In [20]:
# Food Delivery Data Analysis
#GenAI Internship – Final Submission


In [21]:
import pandas as pd
import sqlite3


In [22]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [23]:
users = pd.read_json("users.json")
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [24]:
conn = sqlite3.connect(":memory:")

with open("restaurants.sql", "r") as f:
    sql_script = f.read()

conn.executescript(sql_script)

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [25]:
orders_users = pd.merge(
    orders, users,
    on="user_id",
    how="left"
)

final_df = pd.merge(
    orders_users, restaurants,
    on="restaurant_id",
    how="left"
)

final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [26]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [27]:
# Total orders by Gold members
final_df[final_df["membership"]=="Gold"].shape[0]


4987

In [29]:
final_df["total_amount"].sum()
### Total revenue generated from all orders


np.float64(8011624.12)

In [28]:
final_df.shape[0]


10000

In [31]:
### Average order value by membership
final_df.groupby("membership")["total_amount"].mean()


membership
Gold       797.145556
Regular    805.158434
Name: total_amount, dtype: float64

In [30]:
### Orders by membership type
final_df["membership"].value_counts()


membership
Regular    5013
Gold       4987
Name: count, dtype: int64

In [33]:
### Top 5 cuisines by number of orders
final_df["cuisine"].value_counts().head()


cuisine
Mexican    2581
Italian    2532
Indian     2469
Chinese    2418
Name: count, dtype: int64

In [34]:
### Percentage of orders placed by Gold members
gold_percentage = (
    final_df[final_df["membership"]=="Gold"].shape[0] 
    / final_df.shape[0]
) * 100

round(gold_percentage, 2)


49.87

In [32]:
### Top 5 cities by total revenue
final_df.groupby("city")["total_amount"].sum().sort_values(ascending=False).head()


city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: total_amount, dtype: float64

In [37]:
### Verifying final merged dataset row count
len(final_df)


10000

In [35]:
### Number of orders from restaurants with rating ≥ 4.5
final_df[final_df["rating"] >= 4.5].shape[0]


3374

In [36]:
### City with highest revenue from Gold members
final_df[final_df["membership"]=="Gold"] \
    .groupby("city")["total_amount"] \
    .sum() \
    .idxmax()


'Chennai'

In [39]:
final_df[final_df["membership"] == "Gold"].shape[0]


4987

In [38]:
len(final_df)


10000

In [40]:
round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)


1889367

In [42]:
round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)


np.float64(797.15)

In [44]:
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]


1337

In [41]:
final_df["user_id"].nunique()


2883

In [43]:
final_df[final_df["rating"] >= 4.5].shape[0]


3374